In [27]:
import gzip
import json
import re
import string
import gzip
import pickle
import pandas as pd

filename = './data/ptwiki-latest.json.gz'
clearned_corpus = f'./data/clearned_corpus.txt.gz'
file_word_dict = './data/word_dict.pickle'
file_count_words = './data/count_words.parquet'

regex = r"([" + string.punctuation + '—–¿!' + "])"

In [28]:
samples = 300000
#samples = 5000
maxlen = 40

In [29]:
count = 0
with gzip.open(filename, 'rt') as f:
    with gzip.open(clearned_corpus, 'wt') as f_out:
        for text in f:
            j = json.loads(text)
            for line in j['section_texts']:
                line = re.sub(regex, ' ', line)
                #line = line.replace('\n','')
                while '  ' in line:
                    line = line.replace('  ', ' ')                
                line = line.strip().lower()
                _splited = line.split()
                if len(_splited) > maxlen-1:
                    _splited = _splited[:maxlen-1]
                _splited += ['[end]']
                if len(_splited) < 10:
                    continue               
                for i in range(3, len(_splited)):                    
                    f_out.write(' '.join(_splited[0:i+1]) + '\n')
                if count > samples:
                    break
                count += 1
            if count > samples:
                break

In [30]:
def list_of_words(file: str):
  list_unique_word_file = set()
  list_count_words_file = {}
  with gzip.open(file, 'rt') as f_in:
    lines = f_in.readlines()
    for line in lines:
      for w in line.split():
        list_unique_word_file.add(w)
        if w in list_count_words_file:
          list_count_words_file[w] += 1
        else:
          list_count_words_file[w] = 1
    print(f'Size of Dict for file {file}: {len(list_unique_word_file)}')
  return list_unique_word_file, list_count_words_file

In [ ]:
list_unique_word_file, list_count_words_file = list_of_words(clearned_corpus)

In [ ]:
word_dict = {}
for i, word in enumerate(list_unique_word_file):
  word_dict[i] = word
print('size of word_dict:', len(word_dict))
# Store data (serialize)
with open(file_word_dict, 'wb') as handle:
  pickle.dump(word_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print(f'word_dict save to file: {file_word_dict}')

In [ ]:
df = pd.DataFrame(list_count_words_file.items(), columns=['word', 'count'])
df.info()

In [ ]:
df.sort_values('count', ascending=False).head(100)

In [35]:
df.to_parquet(file_count_words, engine='pyarrow', compression='gzip')